In [1]:

from arch import arch_model
import numpy as np 
import pandas as pd

In [2]:
R = pd. read_csv ('multivariate_time_series.csv', index_col=0,parse_dates=True) 
R.head

<bound method NDFrame.head of                                 IBM       OMC      MCHP        CI       SLB  \
index                                                                         
2000-01-03 05:00:00+00:00  0.067680 -0.031749 -0.004540 -0.026731 -0.023610   
2000-01-04 05:00:00+00:00 -0.030755 -0.033457 -0.077331 -0.003992 -0.013746   
2000-01-05 05:00:00+00:00  0.053744 -0.029105  0.003008 -0.004008 -0.002309   
2000-01-06 05:00:00+00:00 -0.008493  0.005479 -0.021898  0.047068  0.065983   
2000-01-07 05:00:00+00:00 -0.032506  0.041478 -0.026694  0.042751  0.018231   
...                             ...       ...       ...       ...       ...   
2006-12-13 05:00:00+00:00  0.006882  0.005516 -0.009791 -0.006909  0.020091   
2006-12-14 05:00:00+00:00  0.006206  0.004931  0.010971  0.009921  0.019840   
2006-12-15 05:00:00+00:00 -0.001049  0.001512  0.008223 -0.000077 -0.017172   
2006-12-18 05:00:00+00:00  0.001888  0.000472 -0.020686  0.000848 -0.037404   
2006-12-19 05:00:00+00

In [3]:
T_insample = 1500

In [4]:
Rin = R.iloc[: T_insample].values.T 
Rout = R.iloc[T_insample:].values.T

In [5]:
Rin_annual = Rin * 252
ft = arch_model (Rin_annual [0], vol='GARCH',p=1,q=1, dist = 't').fit (disp='off')
ft.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                        Constant Mean - GARCH Model Results                         
====================================================================================
Dep. Variable:                            y   R-squared:                       0.000
Mean Model:                   Constant Mean   Adj. R-squared:                  0.000
Vol Model:                            GARCH   Log-Likelihood:               -4280.65
Distribution:      Standardized Student's t   AIC:                           8571.29
Method:                  Maximum Likelihood   BIC:                           8597.86
                                              No. Observations:                 1500
Date:                      Tue, Jul 15 2025   Df Residuals:                     1499
Time:                              18:48:43   Df Model:                            1
                               Mean Model                               
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
mu            -0.0335  8.456e-02     -0.396      0.692 [ -0.199,  0.132]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0358  2.952e-02      1.213      0.225 [-2.205e-02,9.365e-02]
alpha[1]       0.0288  1.441e-02      1.998  4.576e-02  [5.421e-04,5.703e-02]
beta[1]        0.9679  1.446e-02     66.938      0.000      [  0.940,  0.996]
                              Distribution                              
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
nu             5.4412      0.798      6.815  9.444e-12 [  3.876,  7.006]
========================================================================

Covariance estimator: robust
"""

In [6]:
ret_devolatized = []
last_conditional_volatility = [] 
for i in range (len (Rin_annual)):
    ft = arch_model(Rin_annual[i] , vol= 'GARCH', p=1,q=1, mean= 'Constant', dist = 't').fit(disp='off')
    ret_devolatized.append(ft.resid/ft.conditional_volatility) 
    last_conditional_volatility.append(ft.conditional_volatility[-1])

In [7]:
ret_devolatized = np.array (ret_devolatized) 
last_conditional_volatility = np.diag(last_conditional_volatility)

In [8]:
Correlation_devolatized = np.corrcoef(ret_devolatized)
Covarince_estimator_garch = last_conditional_volatility @ Correlation_devolatized @last_conditional_volatility

In [9]:
Cov_inverse = np.linalg.inv(Covarince_estimator_garch)
weights = Cov_inverse.sum (axis=1) /Cov_inverse.sum()

In [10]:
Rout = np.exp(Rout) - 1

In [11]:
np.sqrt(252)*(weights @Rout) .std()

0.09181620244883354

In [12]:
equally_weighted_volatility = np.sqrt (252)*(Rout.mean(axis=0)).std() 
equally_weighted_volatility

0.1292268021765246

In [13]:
gm_mean_return = 252*(weights @ Rout).mean()
gmv_volatility = np.sqrt(252) * (weights @ Rout).std()
gmv_sharpe_ratio = gm_mean_return/gmv_volatility 
gmv_sharpe_ratio

1.5413481675747214

In [14]:
ew_mean_return = 252 * (Rout. mean (axis=0)).mean ()
ew_volatility = np.sqrt(252) * (Rout.mean(axis=0)).std()
ew_volatility

0.1292268021765246

In [15]:
ew_sharpe_ratio = ew_mean_return / ew_volatility 
ew_sharpe_ratio

0.6834826935636714

In [16]:

Cov_inverse_sample = np.linalg.inv(np.cov(Rin))
weights_sample = Cov_inverse_sample.sum (axis=1) / Cov_inverse_sample.sum ()

sample_volatility = np.sqrt (252)*(weights_sample @ Rout).std()
sample_volatility

0.09445242925031626

In [17]:
sample_sharpe_ratio = np.sqrt (252)*(weights_sample @ Rout).mean()/(weights_sample @ Rout).std()
sample_sharpe_ratio

1.945443039793083